In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F

from models.dataset import *
from models.discriminator import *
from models.generator import *
from models.inverter import *
from gw_loss import *

from models.GAM.src.param_parser import *
from models.GAM.src.gam import *

from tqdm import tqdm
import warnings

from ot.gromov import gromov_wasserstein
from models.args import *

In [2]:
def fxn():
    warnings.warn("deprecated", UserWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

def choose_device():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'

In [3]:
args = Args()
train, labels = get_dataset_with_label(args.graph_type) # entire dataset as train
train_dataset = Graph_sequence_sampler_pytorch(train, labels, args)
train_loader, adj_shape = get_dataloader_labels(train_dataset, args)
noise_dim = args.hidden_size_rnn

GAMachineTrainer = GAMTrainer(args, args.graph_type) # maps from graphs to latent space (of embeddings)
netG = GraphRNN(args=args)
optimizer_trainer, G_optimizer_output, G_scheduler_rnn, G_scheduler_output = netG.init_optimizer(lr=0.1)
optimizer_generator = None

calculating max previous node, total iteration: 12000
iter 0 times
iter 2400 times
iter 4800 times
iter 7200 times
iter 9600 times
max previous node: 10


In [11]:
def trainAE(args, data):
    # TODO: figure out what is "data"

    ### from Barry's code
    X = data['x']
    Y = data['y']
    adj_mat = data['adj_mat']
    label = data['label']
    Y_len = data['len']
    ###

    ###############################################################################################
    # Embedding (GAMTrainer) Update
    ###############################################################################################
    
    GAMachineTrainer.model.train()
    netG.eval()
    for e in range(args.embedding_iters):
        noise = torch.randn(args.batch_size, args.hidden_size_rnn)
        fake_graphs = []
        # for b in range(args.batch_size):
        fake_graph = netG(noise, X, Y, Y_len) # noise[b, :]
        fake_graphs.append(fake_graph)

        # print(fake_graph)
        # TODO: unpad (a.k.a. pack) fake_graphs

        true_graphs = [] # TODO: sample from true dataset, e.g. MUTAG

        # # from process_batch
        # optimizer_trainer.zero_grad()
        # batch_loss = 0
        # for adj in true_graphs:
        #     batch_loss = self.process_graph(
        #         batch_loss=batch_loss, 
        #         already_matrix=True, 
        #         adj=adj, target=0 # may be worth passing classes instead of generic "true" label
        #     )
        # for adj in fake_graphs:
        #     batch_loss = self.process_graph(
        #         batch_loss=batch_loss, 
        #         already_matrix=True, 
        #         adj=adj, target=1
        #     )
        # batch_loss.backward(retain_graph=True)
        # optimizer_trainer.step()
        
    # ###############################################################################################
    # # Generator (GraphRNN) Update
    # ###############################################################################################

    # GAMachineTrainer.model.eval()
    # netG.train()

    # ### generate fake graphs from noise
    # noise = torch.randn(args.generator_batch_size, args.noise_dim)
    # fake_graphs = []
    # for b in range(args.generator_batch_size):
    #     fake_graph = netG(noise[b, :], X, Y, Y_len)
    #     fake_graphs.append(fake_graph)

    # # TODO: unpad (a.k.a. pack) fake_graphs

    # ### compute embeddings of fake_graphs, then pass embeddings to generator for reconstruction
    # recon_fake_graphs = []
    # for fake_adj in fake_graphs:
    #     datadict, features, node = GAMachineTrainer.get_datadict_features_node(adj, target=1) # target unimportant
    #     fake_embedding = GAMachineTrainer.model(
    #         datadict, fake_adj, features, node, get_embedding=True
    #     )
    #     recon_graph = netG(fake_embedding, X, Y, Y_len)
    #     recon_fake_graphs.append(recon_graph)

    # ### do same for true graphs
    # true_graphs = [] # TODO: sample from true dataset, e.g. MUTAG
    
    # # compute embeddings of true_graphs, then pass embeddings to generator for reconstruction
    # recon_true_graphs = []
    # for true_adj in true_graphs:
    #     datadict, features, node = GAMachineTrainer.get_datadict_features_node(adj, target=0)
    #     true_embedding = GAMachineTrainer.model(
    #         datadict, true_adj, features, node, get_embedding=True
    #     )
    #     recon_graph = netG(true_embedding, X, Y, Y_len)
    #     recon_true_graphs.append(recon_graph)

    # batch_gw_loss = 0
    # # compute GW distance on true_graphs/fake_graphs and recon_true_graphs/recon_fake_graphs
    # for adj_o, adj_r in zip(true_graphs, recon_true_graphs):
    #     batch_gw_loss += GWLoss(adj_o, adj_r)
    # for adj_o, adj_r in zip(fake_graphs, recon_fake_graphs):
    #     batch_gw_loss += GWLoss(adj_o, adj_r)
    # batch_gw_loss.backward()
    # optimizer_generator.step()

    return

In [12]:
for d in train_loader:
    trainAE(args, d)
    break

tensor([[[0, 1, 1,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 1],
         [0, 0, 0,  ..., 1, 0, 1],
         [0, 0, 0,  ..., 1, 1, 0]],

        [[0, 1, 1,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 1],
         [0, 0, 0,  ..., 1, 0, 1],
         [0, 0, 0,  ..., 1, 1, 0]],

        [[0, 1, 1,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 1],
         [0, 0, 0,  ..., 1, 0, 1],
         [0, 0, 0,  ..., 1, 1, 0]],

        ...,

        [[0, 1, 1,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 1],
         [0, 0, 0,  ..., 1, 0, 1],
         [0, 0, 0,  ..., 1, 1, 0]],

        [[0, 1, 1,  ..., 0, 0, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [1,